In [34]:
from numba.typed import Dict
from numba import types
from numba import njit, types
from numba.typed import Dict, List

# @njit(fastmath=True)
def check_value_in_previous_pes(PEs: dict, pe_id: int, value: int, inc_current: bool = False) -> bool:
    """
    Check if a given value is present in the NumPy array for a specified processing element (PE).
    If inc_current is True, it also checks in the current PE.

    Args:
    PEs (dict): A dictionary where each key is a PE id and each value is a NumPy array of node indices.
    pe_id (int): The processing element id to check the value against.
    value (int): The value (node index) to check for.
    inc_current (bool): Whether to include the current PE's array in the check.

    Returns:bool: True if the value is found in the specified PE's array, False otherwise.
    """
    
    for id in range(0, pe_id + int(inc_current)):
        array = PEs[id]  # Directly access the array if the key exists
        if value in array:
            return True
    return False  # Return False if the key does not exist

In [35]:
import numpy as np

# Define a dictionary of processing elements (PEs) with NumPy arrays
PEs = {
    0: [10, 20, 30],
    1: [15, 25, 35],
    2: [12, 22, 32]
}

# Define the value to check
value_to_check = 22

# Define the processing element id to check against
pe_id_to_check = 2

# Check if the value is present in the specified PE's array (without including the current PE)
result_without_current = check_value_in_previous_pes(PEs, pe_id_to_check, value_to_check)
print(f"Is {value_to_check} present in PE {pe_id_to_check} (excluding current)? {result_without_current}")

# Check if the value is present in the specified PE's array (including the current PE)
result_with_current = check_value_in_previous_pes(PEs, pe_id_to_check, value_to_check, inc_current=True)
print(f"Is {value_to_check} present in PE {pe_id_to_check} (including current)? {result_with_current}")

Is 22 present in PE 2 (excluding current)? False
Is 22 present in PE 2 (including current)? True
